# Benchmarking different methods

The library provides various evaluation methods for benchmarking methods. Here are some below:

In [ ]:
from tasks.ClassificationTask import ClassificationTask
from recourse_methods.BinaryLinearSearch import BinaryLinearSearch
from models.pytorch_models.SimpleNNModel import SimpleNNModel
from datasets.ExampleDatasets import get_example_dataset
from evaluations.DistanceEvaluator import DistanceEvaluator

dl = get_example_dataset("ionosphere")
dl.default_preprocess()
model = SimpleNNModel(34, [10], 1)

task = ClassificationTask(model, dl)

generator = BinaryLinearSearch(task)

recourses = generator.generate_for_all(neg_value=0)

distance_eval = DistanceEvaluator(task)

distance_eval.evaluate(recourses)